In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE208662"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE208662"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE208662.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE208662.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE208662.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"COPD basal cells are primed towards secretory to multi-ciliated cell imbalance driving increased resilience to environmental stressors"
!Series_summary	"Introduction: Environmental pollutants irritate and injure the bronchial elevator, thereby provoking disease progression in chronic obstructive pulmonary disease (COPD). Epithelial resilience mechanisms to environmental nanoparticles in health and disease are poorly characterized.   Methods: We delineated the impact of prevalent pollutants such as carbon and zinc oxide nanoparticles, on cellular function and progeny in primary human bronchial epithelial cells (pHBEC) from end-stage COPD, early disease and pulmonary healthy individuals. After nanoparticle exposure of pHBECs at the air liquid interface, cell cultures were characterized by  functional assays as well as transcriptome and protein analysis, complemented by single cell analysis in serial samples of  pHBEC culture focussing on basal cell 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background info, this dataset contains gene expression microarray data for COPD studies
is_gene_available = True

# 2.1 Data Availability
# For trait: disease state is available in row 0
trait_row = 0

# No age information is provided in the sample characteristics
age_row = None

# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(x):
    """Convert COPD status to binary: 1 for COPD, 0 for control."""
    if x is None:
        return None
    
    # Extract value after colon if present
    if ":" in x:
        value = x.split(":", 1)[1].strip()
    else:
        value = x.strip()
        
    if "COPD" in value and "control" not in value:
        return 1
    elif "control" in value:
        return 0
    else:
        return None

def convert_age(x):
    """Convert age to numeric value."""
    # Not available in this dataset
    return None

def convert_gender(x):
    """Convert gender to binary: 1 for male, 0 for female."""
    # Not available in this dataset
    return None

# 3. Save Metadata - Initial Filtering
# Trait data is available, so is_trait_available is True
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# For GEO datasets, we would need to load raw matrix files, but we don't have the
# proper file loading code in this step.
# Since we confirmed trait data is available (trait_row is not None),
# we'll note this for the next step that will handle file loading and processing.
print(f"Clinical data is expected to be available with trait_row={trait_row}")
print("Validation step completed successfully. Gene expression data and trait data appear to be available.")
print("Actual data loading and processing will occur in the next step.")


Clinical data is expected to be available with trait_row=0
Validation step completed successfully. Gene expression data and trait data appear to be available.
Actual data loading and processing will occur in the next step.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE208662/GSE208662_series_matrix.txt.gz
Gene data shape: (21448, 32)
First 20 gene/probe identifiers:
Index(['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1',
       'TC0100006480.hg.1', 'TC0100006483.hg.1', 'TC0100006486.hg.1',
       'TC0100006490.hg.1', 'TC0100006492.hg.1', 'TC0100006494.hg.1',
       'TC0100006497.hg.1', 'TC0100006499.hg.1', 'TC0100006501.hg.1',
       'TC0100006502.hg.1', 'TC0100006514.hg.1', 'TC0100006516.hg.1',
       'TC0100006517.hg.1', 'TC0100006524.hg.1', 'TC0100006540.hg.1',
       'TC0100006548.hg.1', 'TC0100006550.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data are not standard human gene symbols
# They appear to be probe IDs (likely from an Affymetrix array or similar platform)
# and will need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'category', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene mapping
# Looking at the data structure, we need:
# - 'ID' column which contains the probe identifiers matching gene_data index (TC0100006437.hg.1, etc)
# - For gene symbols, we need to extract them from the 'SPOT_ID.1' column which contains gene annotations

# 2. Create mapping dataframe
# The 'ID' column in the gene_annotation dataframe contains the same probe IDs as in gene_data.index
# The gene symbols need to be extracted from the 'SPOT_ID.1' column which contains detailed annotations
# We'll use the extract_human_gene_symbols function to parse gene symbols from the annotations

# Create gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'SPOT_ID.1')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First 5 rows of mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)
print(f"Mapped gene expression data shape: {gene_data_mapped.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data_mapped.index[:5])

# Normalize gene symbols to handle gene synonyms
gene_data = normalize_gene_symbols_in_index(gene_data_mapped)
print(f"Final gene expression data shape after normalization: {gene_data.shape}")
print("First 5 normalized gene symbols:")
print(gene_data.index[:5])

# Save the gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (27189, 2)
First 5 rows of mapping:
                  ID                                               Gene
0  TC0100006437.hg.1  NM_001005484 // RefSeq // Homo sapiens olfacto...
1  TC0100006476.hg.1  NM_152486 // RefSeq // Homo sapiens sterile al...
2  TC0100006479.hg.1  NM_198317 // RefSeq // Homo sapiens kelch-like...
3  TC0100006480.hg.1  NM_001160184 // RefSeq // Homo sapiens pleckst...
4  TC0100006483.hg.1  NM_005101 // RefSeq // Homo sapiens ISG15 ubiq...


Mapped gene expression data shape: (85633, 32)
First 5 gene symbols after mapping:
Index(['A-', 'A-1', 'A-2', 'A-52', 'A-E'], dtype='object', name='Gene')
Final gene expression data shape after normalization: (19975, 32)
First 5 normalized gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE208662.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (19975, 32)
Gene data column names (sample IDs):
Index(['GSM6360778', 'GSM6360779', 'GSM6360780', 'GSM6360781', 'GSM6360782'], dtype='object')

Raw clinical data structure:
Clinical data shape: (2, 33)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM6360778', 'GSM6360779', 'GSM6360780',
       'GSM6360781'],
      dtype='object')

Sample characteristics dictionary:
{0: ['disease state: COPD-IV', 'disease state: control'], 1: ['treatment: LPS', 'treatment: Printex', 'treatment: Printex for 72h', 'treatment: Sham', 'treatment: Sham for 72h', 'treatment: Zn']}

Values in trait row:
['!Sample_characteristics_ch1' 'disease state: COPD-IV'
 'disease state: COPD-IV' 'disease state: COPD-IV'
 'disease state: COPD-IV']

Created clinical features dataframe:
Shape: (1, 32)
                                             GSM6360778 GSM6360779 GSM6360780  \
Chronic_obstructive_pulmonary_disease_(COPD)          1          1          1   

                         

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (32, 19976)
For the feature 'Chronic_obstructive_pulmonary_disease_(COPD)', the least common label is '1' with 16 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Chronic_obstructive_pulmonary_disease_(COPD)' in this dataset is fine.



Linked data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE208662.csv
